In [3]:
import pandas as pd
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from networkx.readwrite import json_graph
%matplotlib inline

In [4]:
distances = pd.read_csv("rmsd_cleaned.csv", index_col=0)

In [7]:
sero = pd.read_csv("pid_sero.csv", index_col=0)

In [5]:
mat = np.exp(-1.5 * distances_sorted.values)  # or different factor
norm = np.diag(mat.sum(axis=0)**(-1/3))  # or different factor (interval [-1/2, 0])
matn = np.dot(np.dot(norm, mat), norm)

In [32]:
g = nx.from_numpy_matrix(np.exp(-2 * distances).values)
# g = nx.Graph([(u, v, d) for u, v, d in g.edges(data=True) if d['weight'] > 0.1]) # u != v and
# the above gives not json serializable stuff (??)

In [33]:
sero_nums = {k: i for i, (k, v) in enumerate(sero.sero.value_counts().iteritems())}

In [39]:
for i, p_id in enumerate(distances.index):
    g.node[i] = {'p_id': p_id,
                 'sero': sero.loc[p_id]['sero'],
                 'sero_num': int(sero.loc[p_id]['sero_num'])}

In [40]:
data = json_graph.node_link_data(g)

In [41]:
data['links'] = [link for link in data['links']
                 if link['weight'] > 0.15 and link['source'] != link['target']]

In [43]:
with open('graph.json', 'w') as f:
    json.dump(data, f, indent=4)